In [9]:
import os
import time
import datetime
import requests
import pandas as pd
import numpy as np

In [6]:
clientcode = ""
password = ""
api_key = ""
secret_key = ""

### `User Order Book`

In [4]:
user_order_book_path = f"User_orderbook_{datetime.datetime.now().date()}.csv"

In [12]:
if os.path.exists(user_order_book_path):
    df_Userorder = pd.read_csv(user_order_book_path)
else:
    col = ['Unique_Trade_No','variety','tradingsymbol','symboltoken','transactiontype','exchange','ordertype','producttype','duration','price','squareoff','stoploss','quantity','status','Client_code','time']
    df_Userorder = pd.DataFrame(columns=col)
    df_Userorder.to_csv(user_order_book_path,index=False)

## `Telegram bot` for notification 

In [13]:
def telegram_bot(message):
    x = 0
    while x == 0:
        token = '' # input token 
        userID = '' # input userID
        url = f'https://api.telegram.org/bot{token}/sendMessage'
        data = {'chat_id':userID,'text':message}

        response = requests.post(url,data)
        
        print(response.status_code)
        print(message)
        if response.status_code != 200:
            print("sending again")
            time.sleep(0.5)
            continue
        x = x + 1

## `User Login Models`

In [ ]:
def login(clientcode,password,api_key):
    try:
        data = {
            "clientcode":clientcode,
            "password":password
        }

        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'X-UserType': 'USER',
            'X-SourceID': 'WEB',
            'X-ClientLocalIP': 'CLIENT_LOCAL_IP',
            'X-ClientPublicIP': 'CLIENT_PUBLIC_IP',
            'X-MACAddress': 'MAC_ADDRESS',
            'X-PrivateKey': str(api_key)
        }

        login_url = "https://apiconnect.angelbroking.com/rest/auth/angelbroking/user/v1/loginByPassword"

        req = requests.post(login_url,json=data,headers=headers)
        if req.status_code == 200:
            print(req.json())
            json_data = req.json()
            if json_data['status'] == True:
                return req.json().get('data').get('jwtToken')
            else:

                return json_data
    except Exception as e:
        print(e)

In [ ]:
Access_token = login(clientcode,password,api_key)

In [ ]:
Access_token

## `Please Order`

In [ ]:
headers= { 
    'Authorization': 'Bearer '+str(Access_token), 
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'X-UserType': 'USER', 
    'X-SourceID': 'WEB', 
    'X-ClientLocalIP': 'CLIENT_LOCAL_IP', 
    'X-ClientPublicIP': 'CLIENT_PUBLIC_IP', 
    'X-MACAddress': 'MAC_ADDRESS', 
    'X-PrivateKey': str(api_key)
}

In [ ]:
data = {
    "variety":"NORMAL",
    "tradingsymbol":"SBIN-EQ",
    "symboltoken":"3045",
    "transactiontype":"BUY",
    "exchange":"NSE",
    "ordertype":"MARKET",
    "producttype":"INTRADAY",
    "duration":"DAY",
    "price":"194.50",
    "squareoff":"0",
    "stoploss":"0",
    "quantity":"1"
}

In [ ]:
def place_order(headers,payload):
    try:
        url_angel_order = 'https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/placeOrder'
        response = requests.post(url_angel_order, json = payload, headers=headers)
        if response.status_code == 200:
            df_new_trade=pd.DataFrame(payload.values(),index = payload.keys()).T
            print(df_new_trade)
            print(response.json())
            df_new_trade['order_id'] = int(response.json().get('data').get("orderid"))
            
            df_tradesheet = df_tradesheet.append(df_new_trade,ignore_index = True)
            df_tradesheet.to_csv(user_order_book_path, index=False)
    except Exception as e:
        print(e)

In [ ]:
# place_order(headers,data)

In [ ]:
def update_orderstatus(order_id,status,text):
    
    df = pd.read_csv(user_order_book_path.astype('str')
    if status == "rejected" or status == "cancelled":
        df.loc[(df['order_id'] == order_id),'Algo_checkflag'] = "No"
        
    df.loc[(df['order_id'] == order_id),'Algo_Runcount'] = status
    df.loc[(df['order_id'] == order_id),'Algo_updatetime'] = datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S')
    df.loc[(df['order_id'] == order_id),'reasons'] = text
                     
    

    return df.to_csv(user_order_book_path,index=False)

In [ ]:
start_time = datetime.time(9,15)
end_time = datetime.time(15,45,4)

# `Main Algo`

In [ ]:
while datetime.datetime.now().time() > start_time and  datetime.datetime.now().time() < end_time:
    try:
        time.sleep(1)
        
        df_alluserorder = pd.read_csv(user_order_book_path)
        df_userorder = df_alluserorder[df_alluserorder['Algo_checkflag'] !='No']
        
        if len(df_userorder) > 0:
            df_userorder = df_userorder.reset_index(drop=True)
            order_book_url = "https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/getOrderBook"
            status_req = requesta.get(order_book_url,headers=headers)
            
            if status_req.status_code == 200:
                df_status = pd.read_csv(status_req.json()['data'])
                df_order_update = df_status[df_status['orderid'].isin(df_userorder['order_id'].to_list())]
                df_order_update = df_order_update.reset_index(drop=True)

            for i , row in df_order_update.iterrows():
                if row['orderid'] == check_order['order_id'].iloc[i]:
                    if row['orderstatus'] != checkorder['orderstatus'].iloc[i]:
                        update_orderstatus(str(row['orderid']),row['orderstatus'],row['text'])
                        
                        # Send order to telegram 
                        # msg = str(row.to_dict())
                        # telegram_bot(msg)
                        
    except Exception as e:
        print(e)